# Imports

In [1]:
import numpy as np
import os, sys
import pandas as pd
import glob
from pymongo import MongoClient

print ('Conexao iniciada')
conn = MongoClient()
db = conn.lexit
collection = db.judgements
print ('Conexao OK')

### Create category folders 
#folders = create_label_folders(collection)
#print ('PASTAS OK')

### Store record according category
#export_record_to_category()
#print ('Arquivos OK')

def get_texts_from_judgement(judgement):
    return (judgement['paginaInternaTitulo'] + judgement['ementa'] + judgement['decisao'] + judgement['tese'] + judgement['observacao'] + judgement['doutrina']).replace("\n", "")

Conexao iniciada
Conexao OK


In [2]:
current_folder = os.getcwd()
target_folder = os.path.join(current_folder, '../../db/*.merge.json')
files = glob.glob(target_folder)
print (len(files), files)


10 ['E:\\UFRGS\\Mestrado\\DISSERTACAO\\web-scraper\\src\\ds\\../../db\\_2012.08-2013.merge.json', 'E:\\UFRGS\\Mestrado\\DISSERTACAO\\web-scraper\\src\\ds\\../../db\\_2013-2014.merge.json', 'E:\\UFRGS\\Mestrado\\DISSERTACAO\\web-scraper\\src\\ds\\../../db\\_2014-2015.merge.json', 'E:\\UFRGS\\Mestrado\\DISSERTACAO\\web-scraper\\src\\ds\\../../db\\_2015-2016.merge.json', 'E:\\UFRGS\\Mestrado\\DISSERTACAO\\web-scraper\\src\\ds\\../../db\\_2016-2017.merge.json', 'E:\\UFRGS\\Mestrado\\DISSERTACAO\\web-scraper\\src\\ds\\../../db\\_2017-2018.merge.json', 'E:\\UFRGS\\Mestrado\\DISSERTACAO\\web-scraper\\src\\ds\\../../db\\_2018-2019.merge.json', 'E:\\UFRGS\\Mestrado\\DISSERTACAO\\web-scraper\\src\\ds\\../../db\\_2019-2020.merge.json', 'E:\\UFRGS\\Mestrado\\DISSERTACAO\\web-scraper\\src\\ds\\../../db\\_2020-2021.merge.json', 'E:\\UFRGS\\Mestrado\\DISSERTACAO\\web-scraper\\src\\ds\\../../db\\_2021-2022.08.merge.json']


In [3]:
#import json
ds = []
for row in collection.find():
  ds.append([ row['categoria'], get_texts_from_judgement(row)])  

for i in range(0, 10):
    print(ds[i])

['RE', 'RE 643992 AgREMENTA: AGRAVO REGIMENTAL. RECURSO EXTRAORDINÁRIO PREJUDICADO.    A decisão do Superior Tribunal de Justiça que determina o retorno dos autos à origem para novo julgamento torna prejudicada a análise do recurso extraordinário.    Agravo regimental a que se nega provimento.O Tribunal, por unanimidade e nos termos do voto do Relator, Ministro Joaquim Barbosa (Presidente), negou provimento ao agravo regimental. Ausentes, justificadamente, a Ministra Cármen Lúcia, em participação na Conferencia Internacional - El acceso individual a la justicia Constitucional en América Latina, em Arequipa, Peru, e, neste julgamento, o Ministro Ricardo Lewandowski. Plenário, 29.05.2013.- Acórdão(s) citado(s):(PREJUDICIALIDADE, RECURSO)AI 651966 AgR (2ªT), RE 597267 AgR (1ªT), AI 764057 AgR (1ªT), RE 539785 AgR-ED (2ªT), ARE 681023 ED (1ªT). Número de páginas: 6.Análise: 28/06/2013, AUR.']
['MI', 'MI 1909 AgREMENTA     AGRAVO REGIMENTAL EM MANDADO DE INJUNÇÃO COLETIVO. ART. 40, § 4º, II

In [7]:
import nltk
#nltk.download('punkt')
import string
from collections import defaultdict
from nltk.corpus import stopwords

    
stop_words = set(stopwords.words('portuguese'))
stop_words.add('turma')
stop_words.add('art')
stop_words.add('agr')
#stop_words.add('2ªt')
stop_words.add('-')
#stop_words.add('1ªt-')
stop_words.add('agravo')
print (stop_words)

def clean_text(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.lower()
    return text
                 
def get_tokens(text):
    tokens = nltk.word_tokenize(text, 'portuguese')
    tokens = [t for t in tokens if not t in stop_words]
    return tokens

tokens = defaultdict(list)

for doc in ds:
    doc_label = doc[0]
    doc_text = clean_text(doc[1])
    
    doc_tokens = get_tokens(doc_text)
    
    tokens[doc_label].extend(doc_tokens)

for category_label, category_tokens in tokens.items():
    print (category_label)
    fd = nltk.FreqDist(category_tokens)
    print(fd.most_common(20))

{'estas', 'depois', 'teriam', 'teve', 'fomos', 'estávamos', 'eles', 'pelos', 'estar', 'estivéssemos', 'houvessem', 'um', 'tu', 'estiver', 'minhas', 'formos', 'fora', 'houver', 'tivéssemos', 'estivesse', 'estava', 'estivemos', 'aquele', 'há', 'nossas', 'mais', 'houveria', 'essas', 'era', 'mas', 'meu', 'seus', 'serão', 'dele', 'teríamos', 'teremos', 'houvéramos', 'fossem', 'só', 'temos', 'me', 'ao', 'teria', 'seja', 'tenha', 'tivemos', 'nos', 'tivera', 'estivera', 'esse', 'deles', 'tivermos', 'os', 'tenhamos', 'pelo', 'ela', 'terão', 'tínhamos', 'tem', 'uma', 'foi', 'foram', 'estou', 'das', 'qual', 'suas', 'houveriam', 'estivermos', 'as', 'no', 'nem', 'estivessem', 'fui', 'que', 'estejamos', 'aquela', 'esta', 'para', 'lhe', 'por', 'se', 'nossos', 'eu', 'seu', 'fosse', 'tive', 'não', 'quando', 'aquelas', 'numa', 'seríamos', 'houverem', 'ser', 'fôssemos', 'tivesse', 'estejam', 'tiveram', 'forem', 'estão', 'e', 'houveríamos', 'minha', 'estivéramos', 'pelas', 'quem', 'estes', 'seremos', 'som

In [17]:
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import pickle


def get_splits (docs):
    random.shuffle(docs)
    
    X_train = []
    y_train = []
    
    X_test = []
    y_test = []
    
    pivot = int(.8 * len(docs))
    
    for i in range(0, pivot):
        X_train.append(docs[i][1])
        y_train.append(docs[i][0])
        
    for i in range(pivot, len(docs)):
        X_test.append(docs[i][1])
        y_test.append(docs[i][0])
        
    return X_train, X_test, y_train, y_test

def evaluate_class (title, classifier, vectorizer, X_test, y_test):
    X_test_tfidf = vectorizer.transform(X_test)
    y_pred = classifier.predict(X_test_tfidf)
    
    precision = metrics.precision_score(y_test, y_pred, average='weighted')
    recall = metrics.recall_score(y_test, y_pred, average='weighted')
    f1 = metrics.f1_score(y_test, y_pred, average='weighted')
    
    print("%s\t%f\t%f\t%f\n" % (title, precision, recall, f1))

X_train, X_test, y_train, y_test = get_splits(ds)
vectorizer = CountVectorizer(stop_words=stop_words,ngram_range=(1,3), min_df=3, analyzer='word')

dtm = vectorizer.fit_transform(X_train)

nbc = MultinomialNB().fit(dtm, y_train)

evaluate_class("NAIVE \tTRAIN\t", nbc, vectorizer, X_train, y_train)
evaluate_class("NAIVE \tTEST\t", nbc, vectorizer, X_test, y_test)

C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


NAIVE 	TRAIN		0.827724	0.830270	0.822494

NAIVE 	TEST		0.764417	0.766581	0.753120



C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
